#### Code Outline:

This code is for contour extraction from pots with two bell krater-like handles. By this we mean handles that _stick out_.

Note that we need to tell the algorithm roughly where the handle of the pot lies. The options are:

1. Top: handle starts from the top of the pot e.g phiales, skyphos cuts.
1. Mid: handles that are in the middle-top area, e.g. ollas, siana cups. This can be used for most pots as it's quite a big range.
1. Bot: handles that are roughly in the bottom third of the pot e.g. kalyx kraters.

At present, the algorithm tries to create three contours for each pot, one for "top", "bot" and "mid". But this can easily be changed once the preferred choice is decided on.

In theory, perhaps a .txt or .csv file should be made with labels such as "bot" etc for each pot or pot class.

In [1]:
import matplotlib.pyplot as plt 
import numpy as np
import pylab as pl
import pandas as pd
from math import sqrt
from skimage.filters import threshold_otsu
from skimage import measure
import os
import re
import itertools as it
import csv
from PIL import Image
from skimage.color import rgb2gray
from skimage.filters import gaussian
from skimage.segmentation import active_contour
from skimage import data, img_as_float
from skimage.segmentation import (morphological_chan_vese,
                                  morphological_geodesic_active_contour,
                                  inverse_gaussian_gradient,
                                  checkerboard_level_set)
pl.ion()

### Contour Functions

In [2]:
def remove_handles2(xs,ys,D,loc):

    if loc == "bot":
        ub = np.min(ys) + 7*(np.max(ys)-np.min(ys))/10 #75%  7/10
        lb = np.min(ys) + 4*(np.max(ys)-np.min(ys))/10   #33%  4/10
    if loc == "top":    
        ub = np.min(ys) + 3*(np.max(ys)-np.min(ys))/10   #33%  3/10
        lb = np.min(ys)                               #0%  0
    if loc == "mid":    
        ub = np.min(ys) + 6*(np.max(ys) - np.min(ys))/10 #67%  6/10
        lb = np.min(ys) + 2*(np.max(ys) - np.min(ys))/10   #25%  3/10
    
    
    indents = []
    for i in range(1,len(xs)-1):
        if lb < ys[i] < ub:
            if D == 'R':
                if xs[i] <= xs[i-1] and xs[i] < xs[i+1]:
                    indents.append(i)
            else:
                if xs[i] >= xs[i-1] and xs[i] > xs[i+1]:
                    indents.append(i)
    
    top = indents[-1]
    bot = indents[0]
    
    p1 = min(top,bot)
    p2 = max(top,bot)
    
    # To get the contour without the handle we now cut through the points we think the handle lies.
    xsr = []
    ysr = []
    xsr.extend(xs[:p1])
    ysr.extend(ys[:p1])
    xsr.extend(xs[p2:])
    ysr.extend(ys[p2:])
    
    return xsr,ysr
    

In [3]:
def get_outline_contour(img):
    
    # Get all contours:
    thresh = threshold_otsu(img)
    binary = img > thresh
    cont = measure.find_contours(binary, 0.8)
    
    # Find longest contour:
    cont_ln = []
    for n, contour in enumerate(cont):
        cont_ln.append(len(contour))

    longest_c = sorted(cont_ln,reverse=True)[:20]
    long_ind = []
    for i in range(0,len(cont_ln)):
        if cont_ln[i] in longest_c:
            long_ind.append(i)
            
    k = np.argmax(cont_ln)
            
    return cont[k]

In [4]:
def find_centre(x,y):
    yc = min(y)+(max(y)-min(y))/2
    tnth = max(y)-(max(y)-min(y))/10
    coords = np.where((np.array(y)<tnth) & (np.array(y)>yc)) [0]
    xc = min(np.array(x)[coords])+(max(np.array(x)[coords])-min(np.array(x)[coords]))/2
    return xc,yc,coords

In [5]:
def get_contour_side(X,Y,D):
    
    xc,yc,coords = find_centre(X,Y)    
    
    tnth = min(X) + (max(X)-min(X))/10
    lb = xc - tnth
    ub = xc + tnth
    rng = []
    
    rng = np.where((X >= lb) & (X <=ub))[0]
    
    ktop = np.argmax(Y[rng])
    top_pnt = rng[ktop]
    
    kbot = np.argmin(Y[rng])
    bot_pnt = rng[kbot]

    mx_pnt = max(top_pnt,bot_pnt)
    mn_pnt = min(top_pnt,bot_pnt)
        
    xs1 = list(X[mn_pnt:mx_pnt])
    ys1 = list(Y[mn_pnt:mx_pnt])

    xs2 = list(X)[mx_pnt:] + list(X)[:mn_pnt] 
    ys2 = list(Y)[mx_pnt:] + list(Y)[:mn_pnt] 
    
    if D == "R":
        xs = xs2
        ys = ys2
    else:
        if D == "L":
            xs = xs1
            ys = ys1
        else:
            if min(xs1) > min(xs2):
                D1 = "R"
                D2 = "L"
            else:
                D1 = "L"
                D2 = "R"

            if len(ys1) >= len(ys2):
                ys = ys2
                xs = xs2
                D = D2
            else:
                ys = ys1
                xs = xs1
                D = D1


    return xs,ys,xc,yc,D

In [6]:
def edit_pot_ends(x,y,xs,ys,side):
    
    xc,yc,coords = find_centre(x,y)
    
    if side == "unknown":
        if min(np.array(x)[coords]) < xc:
            side = "L"
        else:
            side = "R"
    
    if side == "L":
        coords2 = np.where(np.array(xs)<xc)[0]
    else:
        coords2 = np.where(np.array(xs)>xc)[0]
        
    xnew = []
    ynew = []
    xnew.append(xc)
    ynew.append(ys[0])
    xnew.extend(np.array(xs)[coords2])
    ynew.extend(np.array(ys)[coords2])
    xnew.append(xc)
    ynew.append(ys[-1])
    
    if side == "L":
        xnew = [(-1*i)+2*xc for i in xnew]

    return xnew,ynew

In [7]:
def smooth_side_contour(x,y,direction):
    
    ys = sorted(y)
    ys = np.unique(np.round(ys))
    y = np.round(y)
    x = np.round(x)
        
    xs = []
    for i in range(0,len(ys)):
        inds = np.where(y==ys[i])
        xy = x[inds]
        if direction == 'R':
            xs.append(max(xy))
        else:
            xs.append(min(xy))
            
    return xs,ys

### Test Contours

In [ ]:
directory = os.fsencode('C:\\Users\\arian\\OneDrive\\Pictures\\pots\\bell_hnd')

direc_fold = 'C:\\Users\\arian\\OneDrive\\Pictures\\pots\\bell_hnd'

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    nme = str(filename)[:-4]
    
    print("\nExtracting from pot: "+nme)

    
    try:

        xcont = []
        ycont = []

        # 1) Find longest contour from oringal pot image. We assume/hope this is the outline of the pot.
        image_grey = data.load(direc_fold+'\\'+str(filename),as_gray=True)
        contour = get_outline_contour(image_grey)
        xcont = contour[:,1]
        ycont = contour[:,0]


        # 2) Get one side of contour. 
        # "L" in the third parameter gets the left side of the pot, "R" the right side, and anything else gets
        # the shortest side.
        xs,ys,xc,yc,D = get_contour_side(xcont,ycont,"Z")


        # 3) Remove handle.
        # "bot" should be used if the handle is the near the bottom of the pot e.g. kalyx kraters.
        # "mid" should be used if the handle is the near the middle-top area, but not right at the top. e.g. bell kraters.
        # "top" should be used for pots where the handle is at the top e.g. phiales.
        # For now, we use all three to compare. 'Try & Excepts' have been put in at this place as a precaution.
        
        try:
            xs1,ys1 = remove_handles2(xs,ys,D,"bot")
        except:
            xs1 = xs
            ys1 = ys
        try:    
            xs2,ys2 = remove_handles2(xs,ys,D,"mid")
        except:
            xs2 = xs
            ys2 = ys
        try:
            xs3,ys3 = remove_handles2(xs,ys,D,"top")
        except:
            xs3 = xs
            ys3 = ys

        # 4) Smooth the contour
        xsmooth1,ysmooth1 = smooth_side_contour(xs1,ys1,D)
        xsmooth2,ysmooth2 = smooth_side_contour(xs2,ys2,D)
        xsmooth3,ysmooth3 = smooth_side_contour(xs3,ys3,D)

        # 5) Edit contour ends so that they start and end at the centre of the pot.
        x1,y1 = edit_pot_ends(xcont,ycont,xsmooth1,ysmooth1,D)
        x2,y2 = edit_pot_ends(xcont,ycont,xsmooth2,ysmooth2,D)
        x3,y3 = edit_pot_ends(xcont,ycont,xsmooth3,ysmooth3,D)


        # 6) Save plot of pots and side contour.
        fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(6, 8))
        axes[0].imshow(image_grey,cmap='gray')
        axes[0].plot(x1,y1,'-r')
        axes[0].title.set_text('Bot -- '+str(len(xs1)))
        axes[1].imshow(image_grey,cmap="gray")
        axes[1].plot(x2,y2,'-r')
        axes[1].title.set_text('Mid -- '+str(len(xs2)))
        axes[2].imshow(image_grey,cmap="gray")
        axes[2].plot(x3,y3,'-r')
        axes[2].title.set_text('Top -- '+str(len(xs3)))
        fig.tight_layout()  
        plt.savefig(nme+"_side_contour.png")
        plt.close()

        print("\nExtracted from pot: "+nme)

        
    except:
        
        print("\nExtraction from pot: "+nme+" unsuccessful")

    

In [9]:
def cont_extraction_2(img_path,hnd_loc):
    
    # hnd_loc can be "bot","mid" or "top", depending on where the handles are roughly located on the pot.

    xcont = []
    ycont = []

    # 1) Find longest contour from oringal pot image. We assume/hope this is the outline of the pot.
    image_grey = data.load(img_path,as_gray=True)
    contour = get_outline_contour(image_grey)
    xcont = contour[:,1]
    ycont = contour[:,0]


    # 2) Get one side of contour. 
    # "L" in the third parameter gets the left side of the pot, "R" the right side, and anything else gets
    # the shortest side.
    xs,ys,xc,yc,D = get_contour_side(xcont,ycont,"Z")


    # 3) Remove handle.
    # "bot" should be used if the handle is the near the bottom of the pot e.g. kalyx kraters.
    # "mid" should be used if the handle is the near the middle-top area, but not right at the top. e.g. bell kraters.
    # "top" should be used for pots where the handle is at the top e.g. phiales.
    # For now, we use all three to compare. 'Try & Excepts' have been put in at this place as a precaution.
    
    xsr,ysr = remove_handles2(xs,ys,D,hnd_loc)


    # 4) Smooth the contour
    xsmooth,ysmooth = smooth_side_contour(xsr,ysr,D)

    # 5) Edit contour ends so that they start and end at the centre of the pot.
    x,y = edit_pot_ends(xcont,ycont,xsmooth,ysmooth,D)

    return image_grey,x,y


    

In [10]:
def cont_extraction_v2(img_path,hnd_loc,cur_dir):
    
    # hnd_loc can be "bot","mid" or "top", depending on where the handles are roughly located on the pot.

    xcont = []
    ycont = []

    # 1) Find longest contour from oringal pot image. We assume/hope this is the outline of the pot.
    
 
    try:
        # 2.1) Get B/W version of original image.
        image_col = data.load(img_path,as_gray=False)
        image_bw = binary_coloured(image_col)
        # 2.2) Create a plot and temporarily save. This will be the new B/W of the pot.
        fig, ax = plt.subplots(figsize=(4, 6))
        ax.imshow(image_bw)
        ax.set_axis_off()
        plt.savefig("pot_new.jpg")
        plt.close()
        
        # 2.3) Get longest contour from new pot image.
        image_grey = data.load(cur_dir+'\\pot_new.jpg',as_gray=True)        

    except:
        # 1) Find longest contour from oringal pot image. We assume/hope this is the outline of the pot.
        image_grey = data.load(img_path,as_gray=True)
    
    #image_grey = data.load(img_path,as_gray=True)
    contour = get_outline_contour(image_grey)
    xcont = contour[:,1]
    ycont = contour[:,0]


    # 2) Get one side of contour. 
    # "L" in the third parameter gets the left side of the pot, "R" the right side, and anything else gets
    # the shortest side.
    xs,ys,xc,yc,D = get_contour_side(xcont,ycont,"Z")


    # 3) Remove handle.
    # "bot" should be used if the handle is the near the bottom of the pot e.g. kalyx kraters.
    # "mid" should be used if the handle is the near the middle-top area, but not right at the top. e.g. bell kraters.
    # "top" should be used for pots where the handle is at the top e.g. phiales.
    # For now, we use all three to compare. 'Try & Excepts' have been put in at this place as a precaution.
    
    xsr,ysr = remove_handles2(xs,ys,D,hnd_loc)


    # 4) Smooth the contour
    xsmooth,ysmooth = smooth_side_contour(xsr,ysr,D)

    # 5) Edit contour ends so that they start and end at the centre of the pot.
    x,y = edit_pot_ends(xcont,ycont,xsmooth,ysmooth,D)

    return image_grey,x,y


    